# Comprehensive EDA Part 7: Case Studies & Compelling Stories

**Objective:** Identify and analyze compelling case studies, extreme events, and success stories to create engaging dashboard narratives.

**Contents:**
1. Extreme Weather Events (2012 Drought, 2019 Floods)
2. Success Stories (Consistent Improvers)
3. Regional Deep Dives (Iowa vs Illinois)
4. What-If Scenarios
5. Compelling Visualizations for Dashboard

**Author:** Ahsan Riaz | CS 245 Machine Learning Project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')

print("="*80)
print("COMPREHENSIVE EDA - PART 7: CASE STUDIES & COMPELLING STORIES")
print("="*80)

COMPREHENSIVE EDA - PART 7: CASE STUDIES & COMPELLING STORIES


In [2]:
df = pd.read_csv('../data/processed/modeling_dataset_final.csv')
print(f"Dataset loaded: {df.shape[0]:,} rows")

Dataset loaded: 82,436 rows


## 1. Extreme Weather Events

### Case Study 1: The 2012 Drought

The 2012 drought was one of the most severe in US history, devastating corn crops across the Midwest.

In [3]:
# Compare 2012 to surrounding years
drought_years = df[df['Year'].isin([2011, 2012, 2013])]

# National average by year
drought_summary = drought_years.groupby('Year')['Yield_BU_ACRE'].agg(['mean', 'std', 'count']).reset_index()

print("2012 DROUGHT IMPACT")
print("="*80)
print(drought_summary.to_string(index=False))

yield_2011 = drought_summary[drought_summary['Year'] == 2011]['mean'].values[0]
yield_2012 = drought_summary[drought_summary['Year'] == 2012]['mean'].values[0]
yield_2013 = drought_summary[drought_summary['Year'] == 2013]['mean'].values[0]

print(f"\n2012 vs 2011: {yield_2012 - yield_2011:.1f} BU/ACRE ({(yield_2012/yield_2011 - 1)*100:.1f}%)")
print(f"2013 Recovery: {yield_2013 - yield_2012:.1f} BU/ACRE ({(yield_2013/yield_2012 - 1)*100:.1f}%)")

# State-level impact
state_drought = drought_years.groupby(['State', 'Year'])['Yield_BU_ACRE'].mean().reset_index()
state_drought_pivot = state_drought.pivot(index='State', columns='Year', values='Yield_BU_ACRE')
state_drought_pivot['Impact_2012'] = state_drought_pivot[2012] - state_drought_pivot[2011]
worst_affected = state_drought_pivot.nsmallest(10, 'Impact_2012')

print("\nWORST AFFECTED STATES (2012 vs 2011)")
print("="*80)
print(worst_affected[['Impact_2012']].to_string())

2012 DROUGHT IMPACT
 Year       mean       std  count
 2011 125.157824 38.412702   1636
 2012 112.791834 41.632901   1690
 2013 147.278815 32.964131   1553

2012 vs 2011: -12.4 BU/ACRE (-9.9%)
2013 Recovery: 34.5 BU/ACRE (30.6%)

WORST AFFECTED STATES (2012 vs 2011)
Year          Impact_2012
State                    
KENTUCKY       -58.910112
ILLINOIS       -58.224158
INDIANA        -47.075990
TENNESSEE      -44.020108
MISSOURI       -39.010450
IOWA           -34.435354
SOUTH DAKOTA   -32.902440
MONTANA        -30.155556
OHIO           -29.263750
WISCONSIN      -26.706746


In [4]:
# Visualization: Before, During, After
top_states = df.groupby('State').size().nlargest(8).index
drought_viz = drought_years[drought_years['State'].isin(top_states)]

fig = px.box(
    drought_viz,
    x='Year',
    y='Yield_BU_ACRE',
    color='State',
    title='2012 Drought Impact Across Major Corn States',
    labels={'Yield_BU_ACRE': 'Yield (BU/ACRE)'},
    height=600
)
fig.show()

### Case Study 2: 2019 Flooding

Excessive rainfall and flooding in 2019 delayed planting and reduced yields.

In [5]:
# 2019 flooding analysis
flood_years = df[df['Year'].isin([2018, 2019, 2020])]
flood_summary = flood_years.groupby('Year')['Yield_BU_ACRE'].agg(['mean', 'std']).reset_index()

print("2019 FLOODING IMPACT")
print("="*80)
print(flood_summary.to_string(index=False))

# Check abandonment rate if available
if 'Area_Planted_ACRES' in df.columns and 'Area_Harvested_ACRES' in df.columns:
    flood_years['Abandonment_Rate'] = 1 - (flood_years['Area_Harvested_ACRES'] / flood_years['Area_Planted_ACRES'])
    abandon_summary = flood_years.groupby('Year')['Abandonment_Rate'].mean()
    print("\nABANDONMENT RATES:")
    print(abandon_summary)

2019 FLOODING IMPACT
 Year       mean       std
 2018 159.238028 39.221535
 2019 154.358254 32.828953
 2020 155.189474 35.516127

ABANDONMENT RATES:
Year
2018    0.127231
2019    0.124836
2020    0.118706
Name: Abandonment_Rate, dtype: float64


## 2. Success Stories

Identify counties with consistent yield improvements.

In [6]:
# Calculate trend for each county
from scipy.stats import linregress

def calculate_trend(group):
    if len(group) < 10:
        return pd.Series({'slope': np.nan, 'r2': np.nan, 'years': len(group)})
    slope, intercept, r, p, se = linregress(group['Year'], group['Yield_BU_ACRE'])
    return pd.Series({'slope': slope, 'r2': r**2, 'years': len(group)})

county_trends = df.groupby(['State', 'County']).apply(calculate_trend).reset_index()
county_trends = county_trends.dropna()

# Top improvers
top_improvers = county_trends[county_trends['years'] >= 15].nlargest(20, 'slope')

print("TOP 20 CONSISTENTLY IMPROVING COUNTIES")
print("="*80)
print(top_improvers[['State', 'County', 'slope', 'r2', 'years']].to_string(index=False))

# Visualize top improver
if len(top_improvers) > 0:
    top_state = top_improvers.iloc[0]['State']
    top_county = top_improvers.iloc[0]['County']
    
    top_data = df[(df['State'] == top_state) & (df['County'] == top_county)].sort_values('Year')
    
    fig = px.scatter(
        top_data,
        x='Year',
        y='Yield_BU_ACRE',
        trendline='ols',
        title=f'Success Story: {top_county}, {top_state}',
        labels={'Yield_BU_ACRE': 'Yield (BU/ACRE)'},
        height=500
    )
    fig.show()

TOP 20 CONSISTENTLY IMPROVING COUNTIES
      State     County    slope       r2  years
      TEXAS      ERATH 5.334058 0.581186   16.0
    ARIZONA       YUMA 4.707615 0.799263   16.0
    GEORGIA     ECHOLS 4.422306 0.790171   24.0
    GEORGIA      DOOLY 4.399471 0.707354   32.0
    GEORGIA     TOOMBS 4.285904 0.763046   36.0
    ARIZONA      PINAL 4.028524 0.596805   23.0
    GEORGIA      BURKE 3.903156 0.804474   43.0
    GEORGIA   TATTNALL 3.888574 0.819179   38.0
    GEORGIA    JENKINS 3.875712 0.753141   38.0
    GEORGIA      CRISP 3.871204 0.637351   31.0
    GEORGIA   COLQUITT 3.868933 0.884811   40.0
    GEORGIA    CALHOUN 3.853292 0.893395   43.0
MISSISSIPPI      ADAMS 3.806537 0.869302   15.0
    ARIZONA   MARICOPA 3.780529 0.722921   26.0
    GEORGIA     TURNER 3.740314 0.806802   39.0
MISSISSIPPI WASHINGTON 3.696437 0.882947   34.0
MISSISSIPPI    SHARKEY 3.652507 0.859605   33.0
    GEORGIA      WORTH 3.643718 0.814356   43.0
MISSISSIPPI    LEFLORE 3.634661 0.906864   37.0
 

## 3. Regional Deep Dive: Iowa vs Illinois

Compare the two largest corn-producing states.

In [7]:
# Iowa vs Illinois comparison
ia_il = df[df['State'].isin(['IOWA', 'ILLINOIS'])]

comparison = ia_il.groupby(['State', 'Year'])['Yield_BU_ACRE'].mean().reset_index()

fig = px.line(
    comparison,
    x='Year',
    y='Yield_BU_ACRE',
    color='State',
    title='Iowa vs Illinois: Yield Comparison Over Time',
    labels={'Yield_BU_ACRE': 'Average Yield (BU/ACRE)'},
    height=500
)
fig.show()

# Summary statistics
summary = ia_il.groupby('State')['Yield_BU_ACRE'].agg(['mean', 'std', 'min', 'max', 'count'])
print("\nIOWA VS ILLINOIS SUMMARY")
print("="*80)
print(summary)


IOWA VS ILLINOIS SUMMARY
                mean        std   min    max  count
State                                              
ILLINOIS  144.573091  39.976585  19.0  246.7   4229
IOWA      149.389140  37.025083  19.1  234.7   4199


## 4. What-If Scenarios

Simulate the impact of different conditions.

In [8]:
# Scenario: What if 2012 drought happened in 2020?
# Compare yield distributions

year_2012 = df[df['Year'] == 2012]['Yield_BU_ACRE']
year_2020 = df[df['Year'] == 2020]['Yield_BU_ACRE']

print("SCENARIO ANALYSIS: 2012 Drought Conditions in 2020")
print("="*80)
print(f"2012 Average Yield: {year_2012.mean():.1f} BU/ACRE")
print(f"2020 Average Yield: {year_2020.mean():.1f} BU/ACRE")
print(f"\nIf 2020 had 2012-level drought:")
print(f"Estimated Impact: -{year_2020.mean() - year_2012.mean():.1f} BU/ACRE")
print(f"Estimated Loss: {(year_2020.mean() - year_2012.mean()) / year_2020.mean() * 100:.1f}%")

# Distribution comparison
fig = go.Figure()
fig.add_trace(go.Histogram(x=year_2012, name='2012 (Drought)', opacity=0.7, nbinsx=40))
fig.add_trace(go.Histogram(x=year_2020, name='2020 (Normal)', opacity=0.7, nbinsx=40))
fig.update_layout(
    title='Yield Distribution: 2012 Drought vs 2020 Normal',
    xaxis_title='Yield (BU/ACRE)',
    yaxis_title='Frequency',
    barmode='overlay',
    height=500
)
fig.show()

SCENARIO ANALYSIS: 2012 Drought Conditions in 2020
2012 Average Yield: 112.8 BU/ACRE
2020 Average Yield: 155.2 BU/ACRE

If 2020 had 2012-level drought:
Estimated Impact: -42.4 BU/ACRE
Estimated Loss: 27.3%
